### U.S. Opiate Prescriptions
Data source:

https://www.kaggle.com/apryor6/us-opiate-prescriptions

Accidental death by fatal drug overdose is a rising trend in the United States. What can you do to help?

This dataset contains summaries of prescription records for 250 common opioid and non-opioid drugs written by 25,000 unique licensed medical professionals in 2014 in the United States for citizens covered under Class D Medicare as well as some metadata about the doctors themselves. This is a small subset of data that was sourced from cms.gov. The full dataset contains almost 24 million prescription instances in long format. I have cleaned and compiled this data here in a format with 1 row per prescriber and limited the approximately 1 million total unique prescribers down to 25,000 to keep it manageable. If you are interested in more data, you can get the script I used to assemble the dataset here and run it yourself. The main data is in prescriber-info.csv. There is also opioids.csv that contains the names of all opioid drugs included in the data and overdoses.csv that contains information on opioid related drug overdose fatalities.

The increase in overdose fatalities is a well-known problem, and the search for possible solutions is an ongoing effort. My primary interest in this dataset is detecting sources of significant quantities of opiate prescriptions. However, there is plenty of other studies to perform, and I am interested to see what other Kagglers will come up with, or if they can improve the model I have already built.

The data consists of the following characteristics for each prescriber

NPI – unique National Provider Identifier number
Gender - (M/F)
State - U.S. State by abbreviation
Credentials - set of initials indicative of medical degree
Specialty - description of type of medicinal practice
A long list of drugs with numeric values indicating the total number of prescriptions written for the year by that individual Opioid.Prescriber - a boolean label indicating whether or not that individual prescribed opiate drugs more than 10 times in the year

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
#%pylab inline
import folium

opioids = pd.read_csv("/kaggle/input/us-opiate-prescriptions/opioids.csv")
presinfo = pd.read_csv("/kaggle/input/us-opiate-prescriptions/prescriber-info.csv")
overdoses = pd.read_csv("/kaggle/input/us-opiate-prescriptions/overdoses.csv")
presinfo.head()

> ### Gender values generally are in this pie below:

In [ ]:
presinfo[presinfo['Gender'].isin(['M','F'])]["Gender"].value_counts().plot(kind='pie',title= "Gender Overall",colors= ['green','black'], table =True)

* ### Gender distribution values more closely

In [ ]:
presinfo.Gender.value_counts()

In [ ]:
presinfo.Gender.value_counts().plot.bar()

### Males who take some drug/ opioids are in the category 1:

In [ ]:
males = presinfo['Opioid.Prescriber'][presinfo['Gender'] == 'M']
males.value_counts()

In [ ]:
males.value_counts().plot.bar()

### Females in the category 1?

In [ ]:
females = presinfo['Opioid.Prescriber'][presinfo['Gender'] == 'F']
females.value_counts() 

In [ ]:
females.value_counts().plot.bar()

### Better look on Gender Prescripsions

In [ ]:
import plotly.express as px

pres = presinfo.groupby(['Gender'], as_index=False)['Opioid.Prescriber'].sum()

fig = px.bar(pres, x='Gender', y='Opioid.Prescriber')
#fig.xticks()
fig.show()

### Specialty Count follows

In [ ]:
pr = presinfo[presinfo['Opioid.Prescriber']==1].groupby('Specialty').count()['State'][:15]
pr.sort_values(ascending=False) 

In [ ]:
pr = presinfo[presinfo['Opioid.Prescriber']==1].groupby('Specialty').count()['State'][:15]

pr.sort_values(ascending=True).plot(
    kind="barh",
    figsize=(12,3),
    title="Kind of Medicine prescribing opiates")

### Other Specialty overviews

In [ ]:
import plotly.express as px
#presinfo[presinfo['Opioid.Prescriber']==1]
spec = presinfo.groupby(['Specialty'], as_index=False)['Opioid.Prescriber'].count().head(40)#Specialty

fig = px.bar(spec, x='Specialty', y='Opioid.Prescriber') 
fig.show()

In [ ]:
spec2 = presinfo.groupby(['Specialty'], as_index=False)['Opioid.Prescriber'].sum()#[:50]#Specialty

fig = px.bar(spec2, x='Specialty', y='Opioid.Prescriber', color='Opioid.Prescriber') 
fig.show()

### Credentials

In [ ]:
presinfo["Credentials"][presinfo["Opioid.Prescriber"] == 1].value_counts(normalize=True)

presinfo["Credentials"].value_counts(normalize=True).head(22)

In [ ]:
presinfo["Credentials"][presinfo["Opioid.Prescriber"] == 1].value_counts(normalize=True)

# Okay seems like MD prescribe the bulk, what's the proportion of mds?
presinfo["Credentials"].value_counts(normalize=True).head(20).plot.bar()

### Let´ look at  states distribution

In [ ]:
presinfo["State"].unique()

In [ ]:
presinfo["State"].value_counts().head(22).plot.bar()

### Lastly we will look at Twelve Top States with Opioids 

In [ ]:
state = presinfo["State"][presinfo["Opioid.Prescriber"] == 1].value_counts(normalize=True)[:12]

state#.head(12)#.value_counts()

In [ ]:
state = presinfo["State"][presinfo["Opioid.Prescriber"] == 1].value_counts(normalize=True)[:12]
state.sort_values(ascending=False).plot(
    kind="bar",
    figsize=(12,3),
    title="Number of US States where opiates are  prescribed the most")

### More interesting overview of the state´s opioids prescription

In [ ]:
import plotly.express as px

state = presinfo.groupby(['State'], as_index=False)['Opioid.Prescriber'].sum() 

fig = px.bar(state, x='State', y='Opioid.Prescriber')#, color='Opioid.Prescriber')
fig.show()

### Below we can see 12 highest prescriptions in the US, 

### too with a location ( in Folium )

In [ ]:
#import folium
map = folium.Map(location=[39.381266, -97.922211],
                        tiles = "Stamen Toner",
                        zoom_start = 4)

folium.Marker([36.778261, -119.417932],
              popup='California',
              icon=folium.Icon(color='red')
             ).add_to(map)

folium.Marker([31.968599, -99.901813], 
              popup='Texas',
              icon=folium.Icon(color='red') 
             ).add_to(map)

folium.Marker([27.664827, -81.515754], 
              popup='Florida',
              icon=folium.Icon(color='red') 
             ).add_to(map)
             # icon=folium.Icon(color='red',icon='bicycle', prefix='fa')

folium.Marker([43.299428, -74.217933], 
              popup='New York',
              icon=folium.Icon(color='red') 
             ).add_to(map)
    
folium.Marker([41.203322, -77.194525], 
              popup='Pennsylvania',
              icon=folium.Icon(color='green',icon='bar-chart', prefix='fa') 
             ).add_to(map)

folium.Marker([40.417287, -82.907123], 
              popup='Ohio',
              icon=folium.Icon(color='lightgreen',icon='bar-chart', prefix='fa') 
             ).add_to(map)

folium.Marker([40.633125, -89.398528], 
              popup='Illinois',
              icon=folium.Icon(color='lightgreen',icon='bar-chart', prefix='fa') 
             ).add_to(map)

folium.Marker([44.314844, -85.602364], 
              popup='Michigan',
              icon=folium.Icon(color='lightgreen',icon='bar-chart', prefix='fa') 
             ).add_to(map)

folium.Marker([35.75957, -79.0193], 
              popup='North Carolina',
              icon=folium.Icon(color='blue',icon='bar-chart', prefix='fa') 
             ).add_to(map)

folium.Marker([32.157435, -82.907123], 
              popup='Georgia',
              icon=folium.Icon(color='blue',icon='bar-chart', prefix='fa') 
             ).add_to(map)

folium.Marker([42.407211, -71.382437], 
              popup='Massachusetts',
              icon=folium.Icon(color='blue',icon='bar-chart', prefix='fa') 
             ).add_to(map)

folium.Marker([47.751074, -120.740139], 
              popup='Washington',
              icon=folium.Icon(color='blue',icon='bar-chart', prefix='fa') 
             ).add_to(map)

#map.add_child(folium.ClickForMarker(popup="World is awesome"))

map